In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
# Define preprocessing function to use in ImageDataGenerator
def preprocess_image(image):
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian Blur
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    # Normalize the image
    blurred_image = blurred_image / 255.0
    # Expand dimensions to match the input shape expected by ResNet, which is 3 channels
    return np.stack((blurred_image,)*3, axis=-1)

In [3]:
# Set up the data generator for training
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_image
)

In [4]:
# Prepare the training generator
train_generator = datagen.flow_from_directory(
    'D:/DATASET/CNN/ballooning/train',  # Adjust this path
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)

Found 7973 images belonging to 2 classes.


In [5]:
# Load ResNet50 base model without the top layer to allow fine-tuning
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

In [6]:
# Freeze all layers in the base model to prevent them from being updated during the first training phase
for layer in base_model.layers:
    layer.trainable = False

In [7]:
# Add custom layers for our specific task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1, activation='sigmoid')(x)  # Sigmoid activation for binary classification

In [8]:
model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
# Train the model
model.fit(train_generator, epochs=5)

Epoch 1/5
250/250 [==============================] - 876s 3s/step - loss: 0.4010 - accuracy: 0.8333
Epoch 2/5
250/250 [==============================] - 951s 4s/step - loss: 0.3175 - accuracy: 0.8531
Epoch 3/5
250/250 [==============================] - 961s 4s/step - loss: 0.2757 - accuracy: 0.8850
Epoch 4/5
250/250 [==============================] - 954s 4s/step - loss: 0.2495 - accuracy: 0.9052
Epoch 5/5
250/250 [==============================] - 955s 4s/step - loss: 0.2304 - accuracy: 0.9131


In [10]:
# Save the model
model.save('D:/DATASET/Models/model_ballooning.h5')

In [ ]:
import json

# Save the training history
with open('D:/DATASET/Models/ballooning_history.json', 'w') as file:
    json.dump(history.history, file)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def preprocess_images(image):
    # Assuming 'image' is loaded as a grayscale array
    # Apply Gaussian Blur to highlight regions of interest
    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
    # Normalize the image
    blurred_image = blurred_image / 255.0
    # Reshape to match the input shape expected by ResNet, which is 3 channels
    return np.stack((blurred_image,)*3, axis=-1)

def load_and_preprocess_image(image_path):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError("Image not found.")
    if image.shape != (299, 299):
        image = cv2.resize(image, (299, 299))  # Resize if not already 299x299
    return preprocess_images(image)

# Predict function with correct handling
def predict_single_image(image_path, model):
    processed_image = load_and_preprocess_image(image_path)
    processed_image = np.expand_dims(processed_image, axis=0)  # Add batch dimension
    probability = model.predict(processed_image)[0]
    score = 0 if probability > 0.5 else 1

    # Load the original image to display
    original_image = cv2.imread(image_path)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert for correct color display
    plt.imshow(original_image)
    plt.title(f'Predicted Ballooning Score: {score}')
    plt.axis('off')
    plt.show()

    return score

In [11]:
# Predict function that handles grayscale images properly
def predict_single_image(image_path, model):
    # Load the image in grayscale directly
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Check if the image is loaded properly and has the expected size
    if image is None or image.shape != (299, 299):
        raise ValueError("Image not found or the size is not 299x299.")
    
    # Preprocess the image
    processed_image = preprocess_image(image)
    processed_image = np.expand_dims(processed_image, axis=0)  # Add batch dimension for prediction
    # Predict the ballooning score
    probability = model.predict(processed_image)[0]
    # Determine the score based on probability
    score = 0 if probability > 0.5 else 1
    # Convert grayscale image back to RGB for display
    display_image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    # Display the image and its predicted score
    plt.imshow(display_image)
    plt.title(f'Predicted Ballooning Score: {score}')
    plt.axis('off')
    plt.show()

    return score

In [12]:
image_path = "D:/DATASET/CNN/ballooning/train/1/8_179_37.png"
score = predict_single_image(image_path, model)

error: OpenCV(4.9.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x59191d0d::Set<1,-1,-1>,struct cv::impl::A0x59191d0d::Set<0,2,5>,4>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1
